### Анализ данных и подготовка к обучению модели. 

In [1]:
import pandas as pd
import numpy as np
import datetime 
import time
from functools import reduce

 __Основные моменты  на которых стоит остановить внимание при предобработке данных:__

__data_train.csv__<br>
Column  Unnamed: 0<br>
значения не привязаны ни к чему, поэтому его можно удалить.<br>
Column  buy_time<br> 
<br>
Предварительно анализируем содержимое даты и времени используем библиотеку datetime. Убеждаемся в том, данный столбец несет информацию только о номере недели, и его преобразуем к номеру недели.<br>

 преобр значение дата/время     день        день недели 0-6         номер недели<br>
 dt_tm: 2018-07-16 01:00:00,	day: 16, 	w_day: 0-Monday,     	week: 29<br>
 dt_tm: 2018-10-15 01:00:00,	day: 15, 	w_day: 0-Monday,     	week: 42<br>
 dt_tm: 2018-12-24 01:00:00,	day: 24, 	w_day: 0-Monday,     	week: 52<br>
 dt_tm: 2018-08-13 01:00:00,	day: 13, 	w_day: 0-Monday,     	week: 33<br>
 dt_tm: 2018-08-06 01:00:00,	day: 6, 	w_day: 0-Monday,     	week: 32<br>
 dt_tm: 2018-08-13 01:00:00,	day: 13, 	w_day: 0-Monday,     	week: 33<br>
 dt_tm: 2018-12-17 01:00:00,	day: 17, 	w_day: 0-Monday,     	week: 51<br>
 dt_tm: 2018-09-24 01:00:00,	day: 24, 	w_day: 0-Monday,     	week: 39<br>
 dt_tm: 2018-12-24 01:00:00,	day: 24, 	w_day: 0-Monday,     	week: 52<br>
 dt_tm: 2018-12-10 01:00:00,	day: 10, 	w_day: 0-Monday,     	week: 50<br>
 dt_tm: 2018-09-10 01:00:00,	day: 10, 	w_day: 0-Monday,     	week: 37<br>
<br>

__data_test.csv__<br>
Выполняем преобразования как и для data_train.csv.<br>

таким образом в трейне и тесте остается 3 информативных столбца: 'id', 'vas_id', 'buy_time'<br>

определим список доступных пользователей для анализа, это id объединенный список test & train.

In [2]:
dt_train = pd.read_csv('cp_data/data_train.csv')

In [3]:
dt_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831653 entries, 0 to 831652
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  831653 non-null  int64  
 1   id          831653 non-null  int64  
 2   vas_id      831653 non-null  float64
 3   buy_time    831653 non-null  int64  
 4   target      831653 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 31.7 MB


In [4]:
dt_test = pd.read_csv('cp_data/data_test.csv')
dt_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71231 entries, 0 to 71230
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  71231 non-null  int64  
 1   id          71231 non-null  int64  
 2   vas_id      71231 non-null  float64
 3   buy_time    71231 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 2.2 MB


In [5]:
# переводит время в номер недели
def user_timestamp_2_week(tstamp):
    return datetime.datetime.fromtimestamp(tstamp).date().isocalendar()[1]

# default Monday
def user_week_to_timestamp( week, year=2018, day=1 ): 
    u_str_dt = datetime.date.fromisocalendar(year, week, day)
    return time.mktime(datetime.datetime.strptime( str(u_str_dt), "%Y-%m-%d").timetuple())

def user_transform_train_data(dt):
    dt.columns= dt.columns.str.strip().str.lower()
    dt['week'] = dt['buy_time'].apply( user_timestamp_2_week)
    dt = dt.astype({'buy_time': 'int32','vas_id': 'int32', 'week': 'int32'})
    dt.drop(labels=['unnamed: 0'], axis=1, inplace=True)
    return dt

In [6]:
dt_target = dt_train['target'].astype(bool)
dt_target.value_counts()

False    771467
True      60186
Name: target, dtype: int64

In [7]:
dt_train.drop( ['target'], axis=1, inplace=True) 

In [8]:
dt_train = user_transform_train_data(dt_train)
dt_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831653 entries, 0 to 831652
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   id        831653 non-null  int64
 1   vas_id    831653 non-null  int32
 2   buy_time  831653 non-null  int32
 3   week      831653 non-null  int32
dtypes: int32(3), int64(1)
memory usage: 15.9 MB


In [9]:
dt_test = user_transform_train_data(dt_test)
dt_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71231 entries, 0 to 71230
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   id        71231 non-null  int64
 1   vas_id    71231 non-null  int32
 2   buy_time  71231 non-null  int32
 3   week      71231 non-null  int32
dtypes: int32(3), int64(1)
memory usage: 1.4 MB


Можно посмотреть распределение по времени на тесте и трайне

In [10]:
print( f' номер недели на train:\n{np.sort(dt_train["week"].unique())}')
print( f' номер недели на test:\n{np.sort(dt_test["week"].unique())}')

 номер недели на train:
[ 1 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
 51 52]
 номер недели на test:
[2 3 4]


Вполне возможно хотят выяснить предысторию успеха!) тест по времени заложен на начало 2-я, 3-я.4-я, недели...<br>
В задании написано:<br>
Данные train и test разбиты по периодам – на train доступно 4 месяцев, а на test отложен последующий месяц... 

Посмотрим список идентификаторов услуг на тесте и трейне

In [11]:
np.sort(dt_train['vas_id'].unique()), np.sort(dt_test['vas_id'].unique())

(array([1, 2, 4, 5, 6, 7, 8, 9], dtype=int32),
 array([1, 2, 4, 5, 6, 7, 8, 9], dtype=int32))

In [13]:
treshold_tstamp = user_week_to_timestamp(30)
treshold_tstamp

1532289600.0

In [35]:
dt_trn_indx = dt_train.loc[ dt_train['buy_time'] >= treshold_tstamp ].index
dt_trn_indx

Int64Index([     0,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            831643, 831644, 831645, 831646, 831647, 831648, 831649, 831650,
            831651, 831652],
           dtype='int64', length=774648)

In [36]:
dt_train.drop( dt_trn_indx, axis=0, inplace=True) 

In [37]:
dt_target.drop( dt_trn_indx, axis=0, inplace=True) 

In [40]:
dt_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57005 entries, 1 to 778442
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   id        57005 non-null  int64
 1   vas_id    57005 non-null  int32
 2   buy_time  57005 non-null  int32
 3   week      57005 non-null  int32
dtypes: int32(3), int64(1)
memory usage: 1.5 MB


In [62]:
len(dt_target)

57005

выявим пользователей из трайна и теста для формирвания датафрейма с фичами. С целью ограничить размер файла фич

In [41]:
common_ids = pd.concat([dt_train, dt_test]).id.unique()
len(common_ids), common_ids

(127040, array([1454121, 2066638, 1913461, ..., 1891350, 2437172,  988236]))

#### Анализ файла с прфилями пользователей features 

Особенность файла, его размер 22GB, моя машина умирает через некоторое время при попытке загрузить данные... <br>

Пробуем варианты... задать chunksize при чтении датафрейма, то что сработало.

In [42]:
def get_features_from_chunk(chunk):
    tmp_df = chunk.loc[( chunk['id'].isin(common_ids) ) & ( chunk['buy_time'] < treshold_tstamp) ].copy()
    return tmp_df

def cumulate_features(prev_res, new_res):    
    return prev_res.append(new_res)

In [43]:
start = time.time()

chunks = pd.read_csv('cp_data/features.csv', sep='\t', nrows=2*16384, chunksize=16384)
processed_chunks = map(get_features_from_chunk, chunks)
result = reduce(cumulate_features, processed_chunks)

stop = time.time()
print(f'elapsed time: {round(stop-start, 3)}\ngot: {len(result)} writes\n', result)

elapsed time: 1.974
got: 78 writes
        Unnamed: 0       id    buy_time           0           1           2  \
497           497  3748015  1531688400  199.810029  282.490888  185.869214   
606           606  3870847  1531083600  335.400029   25.290888  351.119214   
1498         1498   216934  1531083600  -94.679971 -378.949112   14.259214   
1573         1573   267598  1531083600  -96.799971 -196.319112 -110.740786   
2060         2060   893213  1531688400  -96.799971   66.400888  -81.060786   
...           ...      ...         ...         ...         ...         ...   
31341       31341  4068730  1531688400  -52.299971 -233.599112  -66.240786   
31394       31394  4084837  1531688400  -76.469971 -247.839112  -90.410786   
31686       31686  4161464  1531083600  -96.799971 -408.179112 -110.740786   
32414       32414     2750  1531688400  -96.799971  -69.199112 -110.740786   
32578       32578    44693  1531688400  -96.799971 -408.179112 -110.740786   

                3          

Такой подход позволяет отфильтровать нужных пользователей и сохранить в файл... 

In [44]:
chunk_size = 4*16384
num_chunks = 2 

#inp_chunks = pd.read_csv('cp_data/features.csv', sep='\t', nrows=num_chunks*chunk_size, iterator=True, chunksize=chunk_size)
inp_chunks = pd.read_csv('cp_data/features.csv', sep='\t', iterator=True, chunksize=chunk_size)

summ = 0
for idx, chunk in enumerate(inp_chunks):
    start = time.time()
    result= get_features_from_chunk(chunk)
    #print( result.head(2) )
    result.to_csv('cp_data/user_features_by_time.csv', sep='\t', chunksize=chunk_size, mode='a')
    summ += time.time()-start
    print(f'{idx}: elapsed time: {str(datetime.timedelta(seconds=round(summ, 3)))}, got: {len(result)} writes for {round(time.time()-start, 3)} sec')

0: elapsed time: 0:00:00.106000, got: 134 writes for 0.106 sec
1: elapsed time: 0:00:00.183000, got: 116 writes for 0.077 sec
2: elapsed time: 0:00:00.274000, got: 139 writes for 0.091 sec
3: elapsed time: 0:00:00.346000, got: 108 writes for 0.072 sec
4: elapsed time: 0:00:00.432000, got: 132 writes for 0.086 sec
5: elapsed time: 0:00:00.533000, got: 139 writes for 0.101 sec
6: elapsed time: 0:00:00.618000, got: 109 writes for 0.085 sec
7: elapsed time: 0:00:00.691000, got: 109 writes for 0.073 sec
8: elapsed time: 0:00:00.780000, got: 117 writes for 0.089 sec
9: elapsed time: 0:00:00.862000, got: 125 writes for 0.082 sec
10: elapsed time: 0:00:00.961000, got: 128 writes for 0.099 sec
11: elapsed time: 0:00:01.064000, got: 122 writes for 0.103 sec
12: elapsed time: 0:00:01.161000, got: 131 writes for 0.097 sec
13: elapsed time: 0:00:01.256000, got: 128 writes for 0.095 sec
14: elapsed time: 0:00:01.344000, got: 118 writes for 0.088 sec
15: elapsed time: 0:00:01.435000, got: 136 writes 

In [45]:
def get_N_rows_from( file2rd, chunksize=16384, n_chunss=None ):
    
    chunks = pd.read_csv(file2rd, sep='\t', iterator=True, 
                         nrows=n_chunss*chunksize if n_chunss != None else None, 
                         chunksize=chunksize, 
                         low_memory=False
                        )
    res = pd.DataFrame()
    summ=0
    for idx, chnk in enumerate(chunks):
        start = time.time()
        res = res.append(chnk)
        summ += time.time()-start
        print(f'{idx}: elapsed time: {str(datetime.timedelta(seconds=round(summ, 3)))}, \
              got: {len(res)} writes for {round(time.time()-start, 3)} sec')
    return res

def get_N_cols_from( file2rd, columns ):    
    start = time.time()
    res = pd.read_csv(file2rd, sep='\t', usecols=columns)
    stop = time.time()-start
    print(f'Read columns: {columns}\nfrom {file2rd}, elapsed time: {str(datetime.timedelta(seconds=round(stop, 3)))}')
    return res

def get_nunique_in_cols_from( file2rd, col_name ):    
    start = time.time()
    res = pd.read_csv(file2rd, sep='\t', usecols=[col_name,])
    n_unique = { 'unic_cnt':res[col_name].nunique(), 'values':res[col_name].unique() }
    stop = time.time()-start
    print(f'Read column: {col_name}\nfrom {file2rd}, elapsed time: {str(datetime.timedelta(seconds=round(stop, 3)))}')
    return n_unique

In [46]:
feat_1_row = get_N_rows_from('cp_data/user_features_by_time.csv')


0: elapsed time: 0:00:00.010000,               got: 8422 writes for 0.01 sec


In [47]:
bad_indexes = feat_1_row[feat_1_row['id'] == 'id'].index

In [48]:
feat_1_row.drop( labels=bad_indexes, axis=0, inplace=True ) 

In [49]:
feat_1_row.info(), feat_1_row.memory_usage(deep=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8354 entries, 0 to 8421
Columns: 257 entries, Unnamed: 0 to 252
dtypes: float64(254), object(3)
memory usage: 16.4+ MB


(None,
 Index            66832
 Unnamed: 0       66832
 Unnamed: 0.1    532542
 id              532419
 buy_time        559718
                  ...  
 248              66832
 249              66832
 250              66832
 251              66832
 252              66832
 Length: 258, dtype: int64)

In [50]:
feat_1_row.describe()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,243,244,245,246,247,248,249,250,251,252
count,8.354000e+03,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,...,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000,8354.000000
mean,2.249330e+06,5.255322,36.949103,4.047678,31.653667,8.279109,40.167888,3.013243,15.879714,2.910864,...,131.135610,33.916304,0.117689,-10.000810,-8.021365,0.280829,0.085677,12.643435,0.027700,0.268734
std,1.306814e+06,182.462452,375.099163,190.993428,420.104503,216.390235,492.361408,101.210184,262.664089,81.406140,...,3484.015962,2413.461675,958.108001,198.997059,1382.865117,144.948148,6.054197,144.531672,5.710988,0.443328
min,4.970000e+02,-181.509971,-408.179112,-195.450786,-460.786798,-200.868246,-481.891790,-16.086180,-65.076097,-6.783660,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.456140,0.000000
25%,1.091703e+06,-96.799971,-153.939112,-110.740786,-191.064298,-114.888246,-206.351790,-16.086180,-65.076097,-6.783660,...,-977.373846,-613.770792,-25.996269,-37.630448,-294.747724,-25.832889,-0.694428,-12.175933,-0.456140,0.000000
50%,2.251965e+06,-81.959971,-22.579112,-78.960786,-41.296798,-80.598246,-56.031790,-16.086180,-65.076097,-6.783660,...,-937.373846,-610.770792,-25.996269,-37.630448,-219.747724,-22.832889,-0.694428,-10.175933,-0.456140,0.000000
75%,3.377638e+06,21.850029,145.943388,34.851714,151.020702,42.121754,153.698210,-16.086180,-39.656097,-6.783660,...,-334.373846,-351.020792,-24.996269,-32.630448,-53.747724,-8.832889,-0.694428,2.824067,-0.456140,1.000000
max,4.510299e+06,3978.440029,8817.860888,3974.669214,8799.583202,4187.891754,9442.878210,4051.703820,9087.423903,3552.536340,...,65740.626154,61131.229208,83416.003731,7454.369552,93539.252276,6135.167111,168.305572,6148.824067,403.543860,1.000000


In [51]:
feat_1_row.id.nunique(), len(feat_1_row)  

(8347, 8354)

In [52]:
feat_1_row = feat_1_row.astype({'id': 'int32', 'buy_time': 'int32'})

In [53]:
feat_1_row.dtypes

Unnamed: 0      float64
Unnamed: 0.1     object
id                int32
buy_time          int32
0               float64
                 ...   
248             float64
249             float64
250             float64
251             float64
252             float64
Length: 257, dtype: object

In [54]:
feat_1_row.drop( labels=['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True ) 

In [55]:
feat_1_row.info(), feat_1_row.memory_usage(deep=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8354 entries, 0 to 8421
Columns: 255 entries, id to 252
dtypes: float64(253), int32(2)
memory usage: 16.3 MB


(None,
 Index       66832
 id          33416
 buy_time    33416
 0           66832
 1           66832
             ...  
 248         66832
 249         66832
 250         66832
 251         66832
 252         66832
 Length: 256, dtype: int64)

In [56]:
feat_1_row['week'] = feat_1_row['buy_time'].apply( user_timestamp_2_week)
feat_1_row = feat_1_row.astype({'week': 'int32'}) 

In [57]:
feat_1_row.columns[2:-1]

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '243', '244', '245', '246', '247', '248', '249', '250', '251', '252'],
      dtype='object', length=253)

In [58]:
feat_1_row['week'].unique()

array([29, 28], dtype=int32)

In [59]:
col_0_252_types_dict = {}

for cnm in feat_1_row.columns[2:-1]:
    col_0_252_types_dict[cnm]='float32'

feat_1_row = feat_1_row.astype(col_0_252_types_dict)

In [60]:
for cnm in feat_1_row.columns[2:-1]:
    print( f' {cnm}: nunique: { feat_1_row[cnm].nunique() } ')


 0: nunique: 2517 
 1: nunique: 5370 
 2: nunique: 3144 
 3: nunique: 6003 
 4: nunique: 3270 
 5: nunique: 6121 
 6: nunique: 427 
 7: nunique: 1122 
 8: nunique: 628 
 9: nunique: 1282 
 10: nunique: 139 
 11: nunique: 509 
 12: nunique: 101 
 13: nunique: 644 
 14: nunique: 1652 
 15: nunique: 1 
 16: nunique: 2 
 17: nunique: 2 
 18: nunique: 284 
 19: nunique: 355 
 20: nunique: 617 
 21: nunique: 254 
 22: nunique: 16 
 23: nunique: 2 
 24: nunique: 2 
 25: nunique: 158 
 26: nunique: 2 
 27: nunique: 1 
 28: nunique: 11 
 29: nunique: 4 
 30: nunique: 4 
 31: nunique: 2 
 32: nunique: 2 
 33: nunique: 2 
 34: nunique: 182 
 35: nunique: 2 
 36: nunique: 175 
 37: nunique: 713 
 38: nunique: 6958 
 39: nunique: 6538 
 40: nunique: 4698 
 41: nunique: 654 
 42: nunique: 261 
 43: nunique: 5993 
 44: nunique: 5100 
 45: nunique: 5231 
 46: nunique: 872 
 47: nunique: 3815 
 48: nunique: 1167 
 49: nunique: 2566 
 50: nunique: 307 
 51: nunique: 357 
 52: nunique: 444 
 53: nunique:

In [61]:
len( common_ids ), len(feat_1_row) 

(127040, 8354)

In [63]:
dt_train.id

1         1454121
23        2066638
30        1913461
35        2000858
39        1586711
           ...   
778361    3344051
778372    2064912
778374     763706
778378     830354
778442     874976
Name: id, Length: 57005, dtype: int64

In [66]:
len(feat_1_row.id.isin(dt_train.id) ), len(feat_1_row.id.isin(dt_test.id) ), len(feat_1_row)

(8354, 8354, 8354)

In [67]:
feat_1_row.week.unique()

array([29, 28], dtype=int32)

In [70]:
dt_train_mrg = pd.merge( dt_train, feat_1_row, on="id" )

In [77]:
dt_train_mrg.tail(3)

,id,vas_id,buy_time_x,week_x,buy_time_y,0,1,2,3,4,...,244,245,246,247,248,249,250,251,252,week_y
3731,1779513,1,1531083600,28,1531083600,-96.799973,115.120888,-109.240784,85.883202,-114.658249,...,-208.770798,-25.996269,-37.630447,-83.747726,-18.832888,-0.694429,-5.175933,-0.45614,0.0,28
3732,3020876,1,1531688400,29,1531688400,114.220032,84.900887,129.939209,155.173203,124.521751,...,-610.770813,-25.996269,-27.630449,-94.747726,40.167110,-0.694429,53.824066,-0.45614,0.0,29
3733,1757622,2,1531083600,28,1531688400,95.530029,146.040894,98.549217,177.343201,93.131752,...,-613.770813,-23.996269,-37.630447,-58.747723,-18.832888,-0.694429,-5.175933,-0.45614,0.0,29


In [73]:
dt_test_mrg = pd.merge( dt_test, feat_1_row, on="id" )

In [78]:
dt_test_mrg.tail(3)

,id,vas_id,buy_time_x,week_x,buy_time_y,0,1,2,3,4,...,244,245,246,247,248,249,250,251,252,week_y
4685,4088260,2,1548018000,4,1531688400,136.970032,-99.199112,127.269211,-142.486801,121.851753,...,-613.770813,-25.996269,-37.630447,-298.747711,-25.832888,-0.694429,-12.175933,-0.45614,0.0,29
4686,271054,5,1548018000,4,1531688400,132.860031,989.290894,118.919212,953.643188,113.501755,...,-613.770813,-25.996269,-37.630447,-289.747711,-25.832888,-0.694429,-12.175933,-0.45614,1.0,29
4687,2437169,2,1547413200,3,1531083600,-94.679970,-14.979112,-108.620789,-67.586800,-114.038246,...,-613.770813,-21.996269,-37.630447,-255.747726,-18.832888,-0.694429,-5.175933,-0.45614,0.0,28
